## Input Redshift Cluster Endpoint and User

Please input your Amazon Redshift Cluster endpoint and existing database user

In [1]:
REDSHIFT_ENDPOINT = 'redshift-cluster.xxxxxxxxxx.us-east-1.redshift.amazonaws.com:5439/dev'
REDSHIFT_USER="awsuser"

## Setup Run SQL function using Redshift Data API to get SQL query output directly into pandas dataframe

In this step, we are creating function run_sql, which we will use to get SQL query output directly into pandas dataframe. We will also use this function to run DDL statements

In [2]:
import boto3
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

session = boto3.session.Session()
region = session.region_name


def run_sql(sql_text):
    client = boto3.client("redshift-data")
    res = client.execute_statement(Database=REDSHIFT_ENDPOINT.split('/')[1], DbUser=REDSHIFT_USER, Sql=sql_text,
                                   ClusterIdentifier=REDSHIFT_ENDPOINT.split('.')[0])
    query_id = res["Id"]
    while True:
        time.sleep(1)
        status_description = client.describe_statement(Id=query_id)
        status = status_description["Status"]
        if status == "FAILED":
            raise Exception('SQL query failed:' + query_id + ": " + status_description["Error"])
        elif status == "FINISHED":
            if status_description['ResultRows']>0:
                results = client.get_statement_result(Id=query_id)
                column_labels = []
                for i in range(len(results["ColumnMetadata"])): column_labels.append(results["ColumnMetadata"][i]['label'])
                records = []
                for record in results.get('Records'):
                    records.append([list(rec.values())[0] for rec in record])
                df = pd.DataFrame(np.array(records), columns=column_labels)
                return df
            else:
                return query_id

## Create User and Grant Permissions - Optional

As the database adminstrator, you may skip the permissions section. Otherwise, you can create users and grant them permissions with the Principle of Least Privilege in mind.  


If demouser exists with privilege, please revoke before dropping the user:

```sql
revoke all on schema demo_ml from demouser;
```


In [ ]:
permissions_one_sql = """

DROP USER IF EXISTS demouser;

create user demouser with password '<password>';

GRANT CREATE MODEL TO demouser;

"""

for sql_text in permissions_one_sql.split(";")[:-1]:
    run_sql(sql_text);

## Data Preparation 

Data preparation script to be ran on Amazon Redshift

**Note**: Please change `<accountId>` to your AWS Account Id down in the script below


In [3]:
setup_script="""

DROP SCHEMA IF EXISTS DEMO_ML CASCADE;

CREATE SCHEMA DEMO_ML;

DROP TABLE IF EXISTS demo_ml.customer_activity;

CREATE TABLE demo_ml.customer_activity (
state varchar(2), 
account_length int, 
area_code int,
phone varchar(8), 
intl_plan varchar(3), 
vMail_plan varchar(3),
vMail_message int, 
day_mins float, 
day_calls int, 
day_charge float,
total_charge float,
eve_mins float, 
eve_calls int, 
eve_charge float, 
night_mins float,
night_calls int, 
night_charge float, 
intl_mins float, 
intl_calls int,
intl_charge float, 
cust_serv_calls int, 
churn varchar(6),
record_date date);

COPY DEMO_ML.customer_activity 
FROM 's3://redshift-downloads/redshift-ml/customer_activity/' 
IAM_ROLE 'arn:aws:iam::<accountId>:role/RedshiftML' IGNOREHEADER 1 CSV
region 'us-east-1';
"""

## Run data preparation script in Amazon Redshift 

In [4]:
for sql_text in setup_script.strip().split(";")[:-1]:
    run_sql(sql_text);


## Granting Permissions - Optional

Create demo user
Grant create model permissions to `demouser`

In [ ]:
permissions_two_sql = """

GRANT SELECT on demo_ml.customer_activity TO demouser;

GRANT CREATE, USAGE ON SCHEMA demo_ml TO demouser;


"""

for sql_text in permissions_two_sql.split(";")[:-1]:
    run_sql(sql_text);

## Read SQL output from Pandas Dataframe

In [5]:
df = run_sql("SELECT * FROM demo_ml.customer_activity;");
df.head(10)

,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,cust_serv_calls,churn,record_date
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,16.78,244.7,91,11.01,10.0,3,2.7,1,False.,2020-08-24
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,16.62,254.4,103,11.45,13.7,3,3.7,1,False.,2019-09-23
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,10.3,162.6,104,7.32,12.2,5,3.29,0,False.,2020-03-09
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.9,...,5.26,196.9,89,8.86,6.6,7,1.78,2,False.,2019-07-08
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,12.61,186.9,121,8.41,10.1,3,2.73,3,False.,2020-02-14
5,AL,118,510,391-8027,yes,no,0,223.4,98,37.98,...,18.75,203.9,118,9.18,6.3,6,1.7,0,False.,2019-07-28
6,MA,121,510,355-9993,no,yes,24,218.2,88,37.09,...,29.62,212.6,118,9.57,7.5,7,2.03,3,False.,2019-06-07
7,MO,147,415,329-9001,yes,no,0,157.0,79,26.69,...,8.76,211.8,96,9.53,7.1,6,1.92,0,False.,2020-08-22
8,LA,117,408,335-4719,no,no,0,184.5,97,31.37,...,29.89,215.8,90,9.71,8.7,4,2.35,1,False.,2020-04-10
9,WV,141,415,330-8173,yes,yes,37,258.6,84,43.96,...,18.87,326.4,97,14.69,11.2,5,3.02,0,False.,2020-06-06


In [6]:
df.describe()


,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,cust_serv_calls,churn,record_date
count,3333,3333,3333,3333,3333,3333,3333,3333,3333,3333,...,3333,3333,3333,3333,3333,3333,3333,3333,3333,3333
unique,51,212,3,3333,2,2,46,1667,119,1667,...,1440,1591,120,933,162,21,162,10,2,520
top,WV,105,415,389-2540,no,no,0,159.5,102,27.12,...,16.12,191.4,105,9.66,10.0,3,2.7,1,False.,2020-07-18
freq,106,43,1655,1,3010,2411,2411,8,78,8,...,11,8,84,15,62,668,62,1181,2850,16


## Run Create Model statement to create a new ML model with Redshift ML

Please replace `<accountId>` with your AWS account Id

In [8]:
resp = run_sql("""
CREATE MODEL demo_ml.customer_churn_model
  FROM (SELECT state,
               area_code,
               total_charge/account_length AS average_daily_spend, 
               cust_serv_calls/account_length AS average_daily_cases,
               churn 
          FROM demo_ml.customer_activity
         WHERE record_date < '2020-01-01'
     )
  TARGET churn
FUNCTION predict_customer_churn
IAM_ROLE 'arn:aws:iam::<accountId>:role/RedshiftML'
SETTINGS (
  S3_BUCKET 'redshiftml-<accountId>'
)
;
""")

resp

'60d56320-0586-4b4e-bd9e-b0435870b3b8'

## Check the status on your ML model 

You can check the status of your models by running the `SHOW MODEL` command from your SQL prompt.

Continuously check `Model State` and once it has been set to `Ready`, continue to the next step. 

In [10]:
df = run_sql('SHOW MODEL demo_ml.customer_churn_model;')

df.head(10)

,Key,Value
0,Model Name,customer_churn_model
1,Schema Name,demo_ml
2,Owner,awsuser
3,Creation Time,"Mon, 09.08.2021 23:21:38"
4,Model State,READY
5,Training Job Status,MaxAutoMLJobRuntimeReached
6,validation:f1_binary,0.271910
7,Estimated Cost,4.907083
8,,
9,TRAINING DATA:,


## Evaluate your model performance 


In [11]:
df = run_sql("""
WITH infer_data AS (
  SELECT area_code || phone accountid, churn,
    demo_ml.predict_customer_churn( 
          state,
          area_code, 
          total_charge/account_length , 
          cust_serv_calls/account_length ) AS predicted
  FROM demo_ml.customer_activity
 WHERE record_date <  '2020-01-01'
)
SELECT * FROM infer_data where churn!=predicted;
""")
df

,accountid,churn,predicted
0,510355-9993,False.,True.
1,510394-8006,False.,True.
2,510386-2923,False.,True.
3,415373-2782,False.,True.
4,408357-3817,False.,True.
...,...,...,...
406,510380-3186,False.,True.
407,408347-9995,False.,True.
408,510340-9013,False.,True.
409,408362-8378,False.,True.


### Evaluation

You can see the F1 value for the example model customer_churn_model in the output of the `SHOW MODEL` command. The F1 amount signifies the statistical measure of the precision and recall of all the classes in the model. The value ranges between 0–1; the higher the score, the better the accuracy of the model.



In [12]:
df = run_sql('SHOW MODEL demo_ml.customer_churn_model;')
df.head(10)

,Key,Value
0,Model Name,customer_churn_model
1,Schema Name,demo_ml
2,Owner,awsuser
3,Creation Time,"Mon, 09.08.2021 23:21:38"
4,Model State,READY
5,Training Job Status,MaxAutoMLJobRuntimeReached
6,validation:f1_binary,0.271910
7,Estimated Cost,4.907083
8,,
9,TRAINING DATA:,


## Invoke your ML model for inference


You can use your SQL function to apply the ML model to your data in queries, reports, and dashboards. For example, you can run the predict_customer_churn SQL function on new customer data in Amazon Redshift regularly to predict customers at risk of churning and feed this information to sales and marketing teams so they can take preemptive actions, such as sending these customers an offer designed to retain them.

For example, you can run the following query to predict which customers in area code 408 might churn and the output shows the account ID and whether the account is predicted to remain active: 


In [13]:
df = run_sql("""
SELECT area_code || phone accountid, 
       demo_ml.predict_customer_churn( 
          state,
          area_code, 
          total_charge/account_length , 
          cust_serv_calls/account_length )
          AS "predictedActive"
  FROM demo_ml.customer_activity
 WHERE area_code='408' and record_date > '2020-01-01';
""")

df

,accountid,predictedactive
0,408335-4719,True.
1,408350-8884,True.
2,408393-7984,False.
3,408418-6412,True.
4,408383-1121,False.
...,...,...
499,408404-5283,False.
500,408398-3632,False.
501,408340-9449,False.
502,408406-6304,False.


## Granting Permissions - Optional

The following code grants the EXECUTE privilege to users such as your marketing_analyst_grp

In [ ]:
df = run_sql('GRANT EXECUTE demo_ml.predict_customer_churn TO marketing_analyst_grp')

## Cost Control 

If the `SELECT` query of `CREATE MODEL` produces 10,000 records for training and each record has five columns, the number of cells in the training data is 50,000. You can control the training cost by setting the `MAX_CELLS`.

Please replace `<accountId>` with your AWS account Id


In [ ]:
df = run_sql("""
CREATE MODEL demo_ml.customer_churn_model
FROM (SELECT state,
             area_code,
             total_charge/account_length AS average_daily_spend, 
             cust_serv_calls/account_length AS average_daily_cases,
             churn 
      FROM demo_ml.customer_activity
      WHERE account_length > 120 
     )
TARGET churn
FUNCTION predict_customer_churn
IAM_ROLE 'arn:aws:iam::<acountId>:role/RedshiftML'
SETTINGS (
  S3_BUCKET 'redshiftml_<accountId>',
   MAX_CELLS 10000
)
;
""")